# Smart signatures
#### 06.1 Writing Smart Contracts
##### Peter Gruber (peter.gruber@usi.ch)
2022-01-12

* Write and deploy smart Signatures

## Setup
See notebook 04.1, the lines below will always automatically load functions in `algo_util.py`, the five accounts and the Purestake credentials

In [ ]:
# Loading shared code and credentials
import sys, os

codepath = '..'+os.path.sep+'..'+os.path.sep+'sharedCode'
sys.path.append(codepath)
from algo_util import *
cred = load_credentials()

# Shortcuts to directly access the 3 main accounts
MyAlgo  = cred['MyAlgo']
Alice   = cred['Alice']
Bob     = cred['Bob']
Charlie = cred['Charlie']
Dina    = cred['Dina']

In [ ]:
from algosdk import account, mnemonic
from algosdk.v2client import algod
from algosdk import transaction
from algosdk.transaction import PaymentTxn
from algosdk.transaction import AssetConfigTxn, AssetTransferTxn, AssetFreezeTxn
from algosdk.transaction import LogicSig, LogicSigTransaction

import algosdk.error
import json
import base64
import hashlib

In [ ]:
from pyteal import *

In [ ]:
print(Alice['public'])
print(Bob['public'])
print(Charlie['public'])

In [ ]:
# Initialize the algod client (Testnet or Mainnet)
algod_client = algod.AlgodClient(algod_token='', algod_address=cred['algod_test'], headers=cred['purestake_token'])
algod_client.status()["last-round"]

## Using OR in a sensable way
* Either the payment is made to Bob (any amount)
* Or it is maximal 1 Algo (to anybody)

##### Step 1: The programmer writes down the conditions as a PyTeal program

In [ ]:
max_amount = Int(int(1*1E6))                         # <---- 1e6 micro Algos = 1 Algo

escrow_condition_1 =  (
    Txn.receiver() == Addr(Bob["public"])           # Receipient must be Bob
)

escrow_condition_2 =  (
    Txn.amount() <= max_amount                       # Requested amount must be smaller than max_amount
)

# Fancy fee condtion
fee_condition =  (Int(1000) * Txn.fee() <= Int(1000)*Int(1000) + Txn.amount() )  # Max fee 

safety_cond = And(
    Global.group_size() == Int(1), 
    Txn.type_enum() == TxnType.Payment,
    Txn.rekey_to() == Global.zero_address(),
    Txn.close_remainder_to() == Global.zero_address()
    )

modesty2_pyteal = And(
    fee_condition,
    saftey_condition,
    Or(escrow_condition_1,escrow_condition_2)
)

##### Step 2: Compile PyTeal -> Teal

In [ ]:
modesty2_teal = compileTeal(modesty2_pyteal, Mode.Signature, version=8)
print(modesty2_teal)

##### Step 3: Compile Teal -> Bytecode for AVM

In [ ]:
Modesty2 = algod_client.compile(modesty2_teal)
Modesty2

##### Step 4: Alice funds and deploys the smart signature

In [ ]:
# Step 1: prepare transaction
sp = algod_client.suggested_params()
amt = int(5.2*1e6)
txn = transaction.PaymentTxn(sender=Alice['public'], sp=sp, receiver=Modesty2['hash'], amt=amt)

# Step 2+3: sign and sen
stxn = txn.sign(Alice['private'])
txid = algod_client.send_transaction(stxn)

# Step 4: wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

##### Step 5: Alice informs Bob

In [ ]:
print("Alice communicates to Bob the following")
print("Compiled smart signature:", Modesty2['result'])
print("Address of smart signature: ", Modesty2['hash'])

#### Step 6: Charlie makes a small request

In [ ]:
# Step 1: prepare TX
sp = algod_client.suggested_params()
withdrawal_amt = int(1*1e6)              # <---------- OK!!
txn = PaymentTxn(sender=Modesty2['hash'], sp=sp, 
                 receiver=Charlie['public'], amt=withdrawal_amt)

# Step 2: sign TX <---- This step is different!
encodedProg = Modesty2['result'].encode()
program = base64.decodebytes(encodedProg)
lsig = LogicSig(program)
stxn = LogicSigTransaction(txn, lsig)

# Step 3: send
txid = algod_client.send_transaction(stxn)

# Step4: wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

#### Step 6: Dina makes a large request

In [ ]:
# Step 1: prepare TX
sp = algod_client.suggested_params()
withdrawal_amt = int(2*1e6)              # <---------- too much for Dina!!
txn = PaymentTxn(sender=Modesty2['hash'], sp=sp, 
                 receiver=Dina['public'], amt=withdrawal_amt)

# Step 2: sign TX <---- This step is different!
encodedProg = Modesty2['result'].encode()
program = base64.decodebytes(encodedProg)
lsig = LogicSig(program)
stxn = LogicSigTransaction(txn, lsig)


# Step 3: send
try:
    txid = algod_client.send_transaction(stxn)
except algosdk.error.AlgodHTTPError as err:
    print(err)               # print entire error message

# Step 4: wait for confirmation
# No need to wait for confirmation
#txinfo = wait_for_confirmation(algod_client, txid)

#### Step 6: Bob makes a large request
* This is OK, because it is Bob!

In [ ]:
# Step 1: prepare TX
sp = algod_client.suggested_params()
withdrawal_amt = int(2*1e6)              # <---------- ok for Bob!!
txn = PaymentTxn(sender=Modesty2['hash'], sp=sp, 
                 receiver=Bob['public'], amt=withdrawal_amt)

# Step 2: sign TX <---- This step is different!
encodedProg = Modesty2['result'].encode()
program = base64.decodebytes(encodedProg)
lsig = LogicSig(program)
stxn = LogicSigTransaction(txn, lsig)


# Step 3: send
txid = algod_client.send_transaction(stxn)

# Step 4: wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)